In [2]:
!rm -rf Eunomia
!git clone https://github.com/ouaguenouni/Eunomia
!pip install -r Eunomia/requirements.txt

Cloning into 'Eunomia'...
remote: Enumerating objects: 139, done.
remote: Counting objects: 100% (139/139), done.
remote: Compressing objects: 100% (83/83), done.
remote: Total 139 (delta 79), reused 111 (delta 51), pack-reused 0
Receiving objects: 100% (139/139), 326.76 KiB | 953.00 KiB/s, done.
Resolving deltas: 100% (79/79), done.


In [3]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from Eunomia.preferences import *
from Eunomia.additive_functions import *
from Eunomia.alternatives import *
from Eunomia.sampling import *
from Eunomia.mcmc import *
from Eunomia.degree import *
from Eunomia.experiments import *
pyro.set_rng_seed(1)

In [5]:
import time

TEST_NAME = "SX2"
REPLACE = False

n_vals = [5,6,7,8,9,10]
sigma_weights = 10
sigma_w = 1e-1
sigma_p = 1
n_samples = 1500
k=2

for n in n_vals:
    for m in list(2**i for i in range(2, n)):
        run_d = {
            "n": n,
            "k": k,
            "m": m,
            "sigma_w": sigma_w,
            "sigma_p": sigma_p,
            "n_samples": n_samples
        }
        found = find_experiment_file(run_d, TEST_NAME)
        if found:
            date = found
            print(f"Found file: {file_name}")
            if REPLACE:
                print('Replacing it...')
            else:
                continue
        else:
            file_name = compute_experiment_file_name(run_d, TEST_NAME)
            print("Registering the results in: ", file_name)
        theta = generate_additive_theta(n, k)
        weights = generate_normal_weights(theta, sigma_weights)
        alternatives = generate_random_alternatives_matrix(m, n)
        ranks = compute_ws_ranks(alternatives, theta, weights)
        t_sv = compute_semivalues(n, theta, weights, lambda x:1)
        preferences = PreferenceModel(alternatives, ranks)
        data = preferences.generate_preference_matrix(theta)
        data = torch.tensor(data).float()
        t = time.time()
        model = posterior_sampling_model(data, sigma_w=sigma_w, sigma_p=sigma_p)
        diag, sampled_weights, sigmas = sample_model(model, data, "w", "sigma", warmup_steps=500, num_samples=n_samples, return_diag=True)
        t = time.time() - t
        accs_d = get_acc_distribution(data, sampled_weights, sigmas)
        predicted_rankings = [np.argsort(compute_semivalues(n, theta, weights, lambda x:1))[::-1] for weights in sampled_weights]
        kt_d = get_kt_distribution(predicted_rankings, np.argsort(t_sv))
        file_path = record_experiment_results(run_d, TEST_NAME)
        run_d["time"] = t
        run_d["weights"] = weights.tolist()
        run_d["predicted_rankings"] = [i.tolist() for i in predicted_rankings]
        run_d["accuracy_distribution"] = [i.tolist() for i in accs_d]
        run_d["kt_d"] = [i.tolist() for i in kt_d]
        run_d["acceptance_rate"] = diag["acceptance rate"]["chain 0"]
        run_d["w_eff"] = [float(i) for i in diag["w"]["n_eff"].numpy()]
        run_d["w_rhat"] = [float(i) for i in diag["w"]["r_hat"].numpy()]
        print("diag: ", diag)
        with open(file_path, 'w') as file:
            yaml.dump(run_d, file, default_flow_style=False)



Registering the results in:  SX2/2023-11-19/n-5_k-2_m-4_sigma_w-0.1_sigma_p-1_n_samples-1500.yaml


Sample: 100%|████████████████████████████| 2000/2000 [00:26, 76.58it/s, step size=3.93e-01, acc. prob=0.812]


diag:  {'sigma': OrderedDict([('n_eff', tensor(935.7113)), ('r_hat', tensor(0.9993))]), 'w': OrderedDict([('n_eff', tensor([1579.6448, 1502.4774, 1498.2847, 1135.2921, 1225.1014, 1580.6769,
        1277.5460, 1413.5845, 1316.7399,  950.3287, 1527.8398, 1399.8602,
        1302.2622, 1473.0081, 1103.1337])), ('r_hat', tensor([0.9994, 0.9993, 0.9994, 0.9993, 0.9994, 0.9996, 0.9994, 0.9994, 0.9994,
        0.9996, 0.9994, 0.9998, 1.0019, 0.9996, 0.9994]))]), 'divergences': {'chain 0': [25, 28, 93, 130, 239, 246, 250, 251, 315, 358, 365, 443, 446, 451, 505, 527, 528, 531, 532, 695, 746, 762, 801, 804, 916, 925, 944, 967, 980, 1066, 1068, 1069, 1070, 1071, 1072, 1109, 1146, 1147, 1148, 1149, 1150, 1175, 1230, 1248, 1251, 1291, 1298, 1299, 1316, 1383, 1394, 1421, 1446, 1482]}, 'acceptance rate': {'chain 0': 0.9693333333333334}}
Registering the results in:  SX2/2023-11-19/n-5_k-2_m-8_sigma_w-0.1_sigma_p-1_n_samples-1500.yaml


Sample: 100%|████████████████████████████| 2000/2000 [00:58, 34.44it/s, step size=1.64e-01, acc. prob=0.645]


diag:  {'sigma': OrderedDict([('n_eff', tensor(96.3224)), ('r_hat', tensor(1.0017))]), 'w': OrderedDict([('n_eff', tensor([315.0885, 299.9974, 259.3316,  87.3895, 117.8149, 152.1621, 379.3221,
         82.7826, 329.4207, 402.8033, 325.8090, 196.0221, 255.3776, 209.1368,
        218.3308])), ('r_hat', tensor([1.0005, 0.9996, 1.0234, 1.0120, 1.0013, 1.0084, 1.0095, 1.0222, 1.0008,
        1.0059, 0.9998, 1.0025, 1.0000, 0.9994, 1.0097]))]), 'divergences': {'chain 0': [24, 59, 98, 148, 158, 188, 221, 236, 341, 506, 512, 513, 514, 517, 520, 525, 528, 532, 535, 541, 542, 544, 548, 552, 553, 562, 566, 567, 570, 575, 576, 577, 580, 584, 587, 589, 592, 594, 595, 597, 598, 599, 600, 601, 602, 604, 605, 606, 607, 608, 609, 610, 611, 612, 613, 614, 615, 616, 617, 618, 619, 620, 621, 622, 623, 624, 625, 626, 627, 628, 630, 631, 632, 633, 634, 635, 636, 637, 638, 639, 640, 641, 642, 643, 644, 645, 646, 647, 648, 649, 650, 652, 655, 657, 658, 659, 660, 668, 682, 741, 840, 844, 848, 959, 975, 980, 99

...

diag:  {'sigma': OrderedDict([('n_eff', tensor(250.0222)), ('r_hat', tensor(0.9995))]), 'w': OrderedDict([('n_eff', tensor([  48.7141,   66.7065,   49.9490,   44.3862,   49.6193,   59.4960,
          48.1184,   47.0976,   45.8512,   46.6501,   51.1368,  233.3702,
          54.4651,  497.8662,  396.7801,   47.1456,   46.3991,  547.5235,
          47.0666,   70.7775, 1090.3492,   46.0277,   46.0255,  243.7583,
          49.4597,  706.2950,   45.6231,   48.2245,   46.0085,   68.7538,
          62.0028,   69.4447,   45.8536,   63.4757,   48.0737,   59.9416,
         130.6068,  139.0612,  100.8143,  699.3230,   48.5385,  110.6685,
          47.2888,   49.9447,   45.4905,   46.9967,  109.2092,   91.6512,
          60.7330,   62.9787, 1156.9377,  103.0383,   50.7094,  591.7653,
         116.3240])), ('r_hat', tensor([1.0304, 1.0232, 1.0316, 1.0360, 1.0344, 1.0235, 1.0346, 1.0347, 1.0323,
        1.0344, 1.0326, 1.0074, 1.0218, 1.0013, 1.0006, 1.0334, 1.0313, 1.0077,
        1.0348, 1.0180, 0.

Sample: 100%|████████████████████████████| 2000/2000 [20:56,  1.59it/s, step size=3.54e-03, acc. prob=0.839]


diag:  {'sigma': OrderedDict([('n_eff', tensor(62.2899)), ('r_hat', tensor(1.0021))]), 'w': OrderedDict([('n_eff', tensor([  9.9731, 415.5988,  10.0098,  10.1859,  10.5052,   9.8656,   9.7740,
         16.0520,  10.5034,   9.8633,   9.8903,   9.7913,   9.9691,   9.7779,
          9.9055,   9.8956,  10.0482,  11.5221,   9.8910,   9.8568,   9.7974,
          9.9581,   9.9162,   9.8857,   9.9342,   9.8889,  10.0682,   9.8235,
          9.8853,   9.8662,  10.2871,  10.6872,   9.8163,   9.9969,   9.9264,
          9.9007,   9.9642,  10.0695,  10.5276,   9.8694,   9.9892,  11.9329,
          9.9008,  11.2194,   9.9034,  10.0482,   9.9145,   9.9737,   9.8574,
         10.5906, 232.1416,  15.4200,  10.0374,   9.9499,  10.3644])), ('r_hat', tensor([1.2677, 1.0041, 1.2712, 1.2610, 1.2459, 1.2719, 1.2762, 1.1823, 1.2548,
        1.2730, 1.2705, 1.2727, 1.2699, 1.2736, 1.2714, 1.2706, 1.2666, 1.2465,
        1.2705, 1.2725, 1.2767, 1.2676, 1.2704, 1.2706, 1.2695, 1.2751, 1.2642,
        1.2730, 1.

Sample: 100%|██████████████████████████| 2000/2000 [2:07:18,  3.82s/it, step size=1.66e-04, acc. prob=0.780]


diag:  {'sigma': OrderedDict([('n_eff', tensor(5.9040)), ('r_hat', tensor(1.0396))]), 'w': OrderedDict([('n_eff', tensor([ 10.0008,   9.8717,   9.9930,   9.8335,  10.3854,   9.9904,   9.7992,
          9.9968,   9.8008,   9.8591,   9.9484,   9.9028,   9.9146,   9.8189,
          9.8102,   9.8054,   9.7883,   9.8433,  31.3268,   9.8413,   9.8088,
        462.6563,   9.8190,   9.8262,   9.8129,   9.7815,   9.8800,   9.8010,
          9.8504,   9.8079,   9.7734,   9.8534,   9.8270,   9.7976,   9.7982,
          9.8001,   9.8069,   9.8027,   9.8169,   9.8014,   9.8858,   9.7987,
          9.8155,   9.7940,   9.7912,   9.8205,   9.8372,   9.8083,   9.8108,
          9.7898,   9.9331,   9.8006,   9.8118,   9.7794,   9.8081])), ('r_hat', tensor([1.0002, 1.0001, 1.0003, 1.0001, 1.0003, 1.0001, 1.0001, 1.0001, 1.0001,
        1.0001, 1.0004, 1.0002, 1.0000, 1.0003, 1.0002, 1.0002, 1.0002, 1.0001,
        0.9994, 1.0001, 1.0002, 0.9994, 1.0002, 1.0001, 1.0001, 1.0001, 1.0001,
        1.0001, 1.0